In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "1_data_import", ENTITY_NAME: "train_coco_data"},
      { STEP_NAME: "1_data_import", ENTITY_NAME: "eval_coco_data"},
      { STEP_NAME: "1_data_import", ENTITY_NAME: "test_coco_data"},
      { STEP_NAME: "1_data_import", ENTITY_NAME: "config"}
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "aug_dataset" },
        { ENTITY_NAME: "cache_config" },
        { ENTITY_NAME: "cache_data" },
        { ENTITY_NAME: "pretrain_weights"}
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 0_cache_datasets](./imgs/0_cache_datasets.drawio.png)

In [ ]:
#4 get substep.interface
inputs = substep.inputs(step_name = "1_data_import")
outputs = substep.outputs()
tmp_outputs = substep.tmp_outputs()

print(f"{inputs.train_coco_data=}")
print(f"{inputs.eval_coco_data=}")
print(f"{inputs.test_coco_data=}")
print(f"{inputs.config=}")

print(f"{tmp_outputs.aug_dataset=}")
print(f"{tmp_outputs.cache_config=}")
print(f"{tmp_outputs.cache_data=}")
print(f"{tmp_outputs.pretrain_weights=}")

In [ ]:
# Create dir by URLs of tmp_outputs
import os
os.makedirs(tmp_outputs.aug_dataset, exist_ok=True)
os.makedirs(tmp_outputs.cache_config, exist_ok=True)
os.makedirs(tmp_outputs.cache_data, exist_ok=True)
os.makedirs(tmp_outputs.pretrain_weights, exist_ok=True)

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

#### Loading processed datasets and configuration files (from the previous component 1_data_import)

In [ ]:
import os.path as osp
import os
from sinara.store import SinaraStore
from pathlib import Path

# copy config from previos step to outputs
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.config, tmp_dir=tmp_outputs.cache_config)
# Create SUCCESS file after copy from store
Path(osp.join(tmp_outputs.cache_config, '_SUCCESS')).touch()

In [ ]:
### load dataset from parquet
def save_file(row): 
    total_dir_name = tmp_outputs.aug_dataset
    total_img_path = osp.join(total_dir_name, row.file_names)
    os.makedirs(total_dir_name, exist_ok=True)
    with open(total_img_path, 'wb') as f_id:
        f_id.write(row.files_binary)

In [ ]:
%%time

# CACHE TEST

print(f"spark read start")

df_spark = spark.read.parquet(inputs.test_coco_data)
df_spark.foreach(save_file)

In [ ]:
%%time

# CACHE VALID

print(f"spark read start")

df_spark = spark.read.parquet(inputs.eval_coco_data)
df_spark.foreach(save_file)

In [ ]:
%%time

# CACHE TRAIN

print(f"spark read start")

df_spark = spark.read.parquet(inputs.train_coco_data)
df_spark.foreach(save_file)

In [ ]:
# Create SUCCESS file
Path(osp.join(tmp_outputs.aug_dataset, '_SUCCESS')).touch()
Path(osp.join(tmp_outputs.cache_data, '_SUCCESS')).touch()
Path(osp.join(tmp_outputs.pretrain_weights, '_SUCCESS')).touch()

In [ ]:
#9 stop spark
SinaraSpark.stop_session()